In [1]:
from datasets import load_dataset, load_from_disk
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
import aiohttp

Only uncomment loading and saving if we want to overwrite dataset.hf

In [2]:
# dataset = load_dataset("open_subtitles", lang1="en", lang2="es", trust_remote_code=True,
#           storage_options={'client_kwargs': {'timeout': aiohttp.ClientTimeout(total=3600)}}, num_proc= 8, split=None)

In [3]:
#dataset.save_to_disk("dataset.hf")

In [4]:
dataset = load_from_disk("dataset.hf")
split_dataset = dataset["train"].train_test_split(train_size=0.8, seed=20)

In [5]:
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="es_XX")

In [6]:
max_length = 512

def preprocess_text(text):
    inputs = [txt["en"] for txt in text["translation"]]
    targets = [txt["es"] for txt in text["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [7]:
tokenized_datasets = split_dataset.map(
    preprocess_text,
    batched=True,
    remove_columns=split_dataset["train"].column_names,
)

Map:   0%|          | 0/49147400 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
tokenized_datasets.save_to_disk("tokenized_dataset.hf")